## Instalações e Imports

In [ ]:
!pip install scikit-fuzzy
!pip install scipy


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from scipy import signal


## SCI
Desenvolva aqui o código do seu Sistema de Controle Inteligente Fuzzy.
Lembre-se de que o nome do Control System Simulation deve ser "motorcc".

In [ ]:
# Lembre-se de que o nome do Control System Simulation deve ser "motorcc": motorcc = ctrl.ControlSystemSimulation(motorcc_ctrl)

## Inferência
A entrada deve ser uma simulação de controle com o nome "motorcc"

In [ ]:
# Constantes do motor

pi = math.pi

Pn = 7457 # Potência nominal (W)
Vn = 240  # Tensão nominal de armadura (V)
Vfn = 300 # Tensão nominal de campo (V)
wn = 1750 * 2 * pi / 60 # velocidade angular nominal (rad/s)
Tn = Pn / wn # Torque nominal (Nm)

Ra = 1.086    # Resistência do armadura (Ohms)
La = 0.01216  # Indutância do armadura (H)
Rf = 180
Lf = 71.47
Laf = 0.6458 # Este é o Kv da simulação matlab
J = 0.04251   # Inércia do rotor (kg.m^2)
B = 0.003406    # Coeficiente de atrito viscoso (Nms)
Tf = 1.046

If = Vfn / Rf
Ke = Laf*If
Kt = Ke


In [ ]:
# Parâmetros de simulação
dt = 1e-2  # Passo de tempo (s)
t_sim = 10

# Inicialização
t = np.arange(0, t_sim, dt)
i = np.zeros_like(t)
w = np.zeros_like(t)

max = np.ones_like(t) * 200
min = np.ones_like(t) * 180

# Condições iniciais
i[0] = 0
w[0] = 100

# Gerar ruído aleatório
ruido = np.random.rand(len(t))

# Criar um filtro passa-baixa Butterworth
fc = 2  # Frequência de corte (ajuste para controlar a suavidade)
ordem = 4  # Ordem do filtro (ajuste para controlar a suavidade)
b, a = signal.butter(ordem, fc * dt * 2, 'lowpass')

T_minimo = 0.1 * Tn  # Valor mínimo de torque
T_suave = signal.filtfilt(b, a, ruido) * Tn + T_minimo

# Plotar o ruído original e o ruído suavizado
plt.title('Torque de Entrada')
plt.plot(t, T_suave)
plt.show()

# Loop de simulação
for k in range(len(t) - 1):
    T = T_suave[k]

    motorcc.input['Velocidade Angular'] = w[k]
    motorcc.input['Torque da Carga'] = T
    motorcc.compute()

    V = motorcc.output['Tensão']

    # Calcular a corrente e a velocidade no próximo passo de tempo
    i[k+1] = i[k] + dt * (V - Ra*i[k] - Ke*w[k]) / La
    w[k+1] = w[k] + dt * (Kt*i[k] - T - B*w[k]) / J

# Plotar os resultados
plt.title('Velocidade Controlada')
plt.plot(t, max, color = 'red', )
plt.plot(t, min, color = 'red')
plt.plot(t, w)
plt.show()